<a href="https://colab.research.google.com/github/romaforsal/Entornos/blob/main/Keytool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%writefile pykeytool.py
# keeytool_sim.py
import argparse
import json
import base64
import os
from getpass import getpass
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography import x509
from cryptography.x509.oid import NameOID

# ============================================
# Función para pedir contraseña con confirmación y mínimo 6 caracteres
# ============================================
def pedir_contrasena(mensaje: str) -> str:
    while True:
        p1 = getpass(mensaje)
        if len(p1) < 6:
            print("❌ La contraseña debe tener al menos 6 caracteres.\n")
            continue
        p2 = getpass("Vuelve a introducir la contraseña: ")
        if p1 != p2:
            print("❌ Las contraseñas no coinciden. Intenta de nuevo.\n")
        else:
            return p1

# ============================================
# Comando: genkey
# ============================================
def genkey(args):
    keystore_file = args.keystore
    alias = args.alias
    dname = args.dname or f"CN={alias}"

    # Cargar o crear keystore
    if os.path.exists(keystore_file):
        with open(keystore_file, "r") as f:
            keystore = json.load(f)
        print(f"📦 Keystore '{keystore_file}' cargado.")
        ks_pass_input = getpass("Introduce la contraseña del keystore: ")
        stored_ks_pass = base64.b64decode(keystore.get("_keystore_password", "").encode()).decode()
        if ks_pass_input != stored_ks_pass:
            print("❌ Contraseña del keystore incorrecta.")
            return
        ks_pass = ks_pass_input
    else:
        print(f"📦 Creando nuevo keystore: '{keystore_file}'")
        ks_pass = pedir_contrasena("Crea una contraseña para el keystore: ")
        keystore = {}  # Inicializar nuevo keystore

    # Contraseña de la clave
    key_pass = pedir_contrasena("Introduce la contraseña de la clave privada: ")

    # Encriptar contraseñas en Base64
    ks_pass_enc = base64.b64encode(ks_pass.encode()).decode()
    key_pass_enc = base64.b64encode(key_pass.encode()).decode()

    # Generar par de claves RSA
    print("🔑 Generando par de claves RSA...")
    private_key = rsa.generate_private_key(public_exponent=65537, key_size=args.keysize)
    public_key = private_key.public_key()

    # Convertir a PEM y Base64
    priv_b64 = base64.b64encode(
        private_key.private_bytes(
            serialization.Encoding.PEM,
            serialization.PrivateFormat.TraditionalOpenSSL,
            serialization.NoEncryption()
        )
    ).decode("utf-8")

    pub_b64 = base64.b64encode(
        public_key.public_bytes(
            serialization.Encoding.PEM,
            serialization.PublicFormat.SubjectPublicKeyInfo
        )
    ).decode("utf-8")

    # Guardar en keystore
    keystore["_keystore_password"] = ks_pass_enc
    keystore[alias] = {
        "dname": dname,
        "key_password": key_pass_enc,
        "private_key": priv_b64,
        "public_key": pub_b64
    }

    with open(keystore_file, "w") as f:
        json.dump(keystore, f, indent=4)

    print(f"✅ Clave '{alias}' generada y almacenada correctamente en '{keystore_file}'.")

# ============================================
# Comando: certreq
# ============================================
def certreq(args):
    keystore_file = args.keystore
    alias = args.alias

    if not os.path.exists(keystore_file):
        print("❌ El keystore no existe.")
        return

    with open(keystore_file, "r") as f:
        keystore = json.load(f)

    # Verificar contraseña del keystore
    ks_pass_input = getpass("Introduce la contraseña del keystore: ")
    stored_ks_pass = base64.b64decode(keystore["_keystore_password"]).decode()
    if ks_pass_input != stored_ks_pass:
        print("❌ Contraseña del keystore incorrecta.")
        return

    if alias not in keystore:
        print("❌ Alias no encontrado en el keystore.")
        return

    # Verificar contraseña de la clave
    key_pass_input = getpass("Introduce la contraseña de la clave privada: ")
    stored_key_pass = base64.b64decode(keystore[alias]["key_password"]).decode()
    if key_pass_input != stored_key_pass:
        print("❌ Contraseña de la clave incorrecta.")
        return

    # Recuperar la clave privada
    priv_pem = base64.b64decode(keystore[alias]["private_key"].encode("utf-8"))
    private_key = serialization.load_pem_private_key(priv_pem, password=None)

    # Crear CSR
    subject = x509.Name([x509.NameAttribute(NameOID.COMMON_NAME, keystore[alias]["dname"])])
    csr = x509.CertificateSigningRequestBuilder().subject_name(subject).sign(private_key, hashes.SHA256())

    out_file = args.out or f"{alias}.csr.pem"
    with open(out_file, "wb") as f:
        f.write(csr.public_bytes(serialization.Encoding.PEM))

    print(f"✅ CSR generado correctamente y guardado en '{out_file}'.")

# ============================================
# Comando: delete
# ============================================
def delete_keystore(args):
    keystore_file = args.keystore
    if not os.path.exists(keystore_file):
        print("❌ El keystore no existe.")
        return

    confirm = input(f"⚠️ Esto eliminará el keystore '{keystore_file}' de forma permanente.\n¿Estás seguro? (s/n): ").strip().lower()
    if confirm == "s":
        os.remove(keystore_file)
        print("✅ Keystore eliminado correctamente.")
    else:
        print("❎ Operación cancelada.")

# ============================================
# Función de ayuda estilo keytool
# ============================================
def print_help(command=None):
    if not command:
        print("""
keeytool [command] [options]

Commands:
  genkey      Generar un par de claves RSA y guardarlas en un keystore
  certreq     Generar un CSR desde un keystore existente
  delete      Eliminar el keystore indicado
  help        Mostrar esta ayuda
Use "keeytool <command> --help" for command-specific options.
""")
    elif command == "genkey":
        print("""
keeytool -genkey [OPTION]...

Generates a key pair

Options:
 -alias <alias>        alias name of the entry to process (default: mykey)
 -keysize <size>       key bit size (default: 2048)
 -dname <name>         distinguished name (default: CN=<alias>)
 -keystore <keystore>  keystore name (default: keystore.json)
 -h, --help            show this help message and exit
""")
    elif command == "certreq":
        print("""
keeytool -certreq [OPTION]...

Generates a Certificate Signing Request (CSR)

Options:
 -alias <alias>        alias name of the entry to process (default: mykey)
 -keystore <keystore>  keystore name (default: keystore.json)
 -keypass <arg>        key password (interactive)
 -storepass <arg>      keystore password (interactive)
 -out <file>           output file for CSR (default: <alias>.csr.pem)
 -h, --help            show this help message and exit
""")
    elif command == "delete":
        print("""
keeytool -delete [OPTION]...

Deletes a keystore file

Options:
 -keystore <keystore>  keystore name (default: keystore.json)
 -h, --help            show this help message and exit
""")

# ============================================
# CLI principal
# ============================================
def main():
    parser = argparse.ArgumentParser(add_help=False)
    parser.add_argument("--genkey", action="store_true")
    parser.add_argument("--certreq", action="store_true")
    parser.add_argument("--delete", action="store_true")
    parser.add_argument("--keystore", default="keystore.json")
    parser.add_argument("--alias", default="mykey")
    parser.add_argument("--keysize", type=int, default=2048)
    parser.add_argument("--dname")
    parser.add_argument("--out")
    parser.add_argument("-h", "--help", action="store_true")

    args = parser.parse_args()

    # Mostrar ayuda general o de comando específico
    if args.help:
        if args.genkey:
            print_help("genkey")
        elif args.certreq:
            print_help("certreq")
        elif args.delete:
            print_help("delete")
        else:
            print_help()
        return

    # Ejecutar comandos
    if args.genkey:
        genkey(args)
    elif args.certreq:
        certreq(args)
    elif args.delete:
        delete_keystore(args)
    else:
        print_help()

if __name__ == "__main__":
    main()


Overwriting pykeytool.py


Comando sin argumentos

In [5]:
!python pykeytool.py


keeytool [command] [options]

Commands:
  genkey      Generar un par de claves RSA y guardarlas en un keystore
  certreq     Generar un CSR desde un keystore existente
  delete      Eliminar el keystore indicado
  help        Mostrar esta ayuda
Use "keeytool <command> --help" for command-specific options.



Help para comando especifico


In [6]:
!python pykeytool.py --genkey --help


keeytool -genkey [OPTION]...

Generates a key pair

Options:
 -alias <alias>        alias name of the entry to process (default: mykey)
 -keysize <size>       key bit size (default: 2048)
 -dname <name>         distinguished name (default: CN=<alias>)
 -keystore <keystore>  keystore name (default: keystore.json)
 -h, --help            show this help message and exit



Generar un par de claves

In [9]:
!python pykeytool.py --genkey --keystore my_keystore2.json --alias mykey --dname "CN=example.com" --keysize 2048

📦 Creando nuevo keystore: 'my_keystore2.json'
Crea una contraseña para el keystore: 
❌ La contraseña debe tener al menos 6 caracteres.

Crea una contraseña para el keystore: 
Vuelve a introducir la contraseña: 
❌ Las contraseñas no coinciden. Intenta de nuevo.

Crea una contraseña para el keystore: Traceback (most recent call last):
  File "/content/pykeytool.py", line 248, in <module>
  File "/content/pykeytool.py", line 239, in main
    genkey(args)
  File "/content/pykeytool.py", line 48, in genkey
    ks_pass = pedir_contrasena("Crea una contraseña para el keystore: ")
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/pykeytool.py", line 17, in pedir_contrasena
    p1 = getpass(mensaje)
         ^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/getpass.py", line 77, in unix_getpass
    passwd = _raw_input(prompt, stream, input=input)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/getpass.py", line 146, in _raw_inp

Generar la Solicitud de Firma de Certificado (CSR).

In [11]:
!python pykeytool.py --certreq --keystore my_keystore.json --alias mykey --out mykey.csr.pem

Introduce la contraseña del keystore: 
Introduce la contraseña de la clave privada: 
✅ CSR generado correctamente y guardado en 'mykey.csr.pem'.


Borrar el keystore permanentemente

In [13]:
!python pykeytool.py --delete --keystore my_keystore.json

⚠️ Esto eliminará el keystore 'my_keystore.json' de forma permanente.
¿Estás seguro? (s/n): s
✅ Keystore eliminado correctamente.
